In [4]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input
import joblib
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
#importing needed elements 
model1 = tf.keras.models.load_model('../models/resnet50_model_3/')
model2 = tf.keras.models.load_model('../models/resnet50_model_4/')
model3 = tf.keras.models.load_model('../models/resnet50_model_5/')
train_data_dir = ('../images/train/')
val_data_dir = ('../images/val/')
test_data_dir = ('../images/test/')
batch_size = 32
num_classes = 4
input_shape = (224, 224, 3)

In [6]:
#preprocess for training data 
train_data_gen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_data_gen.flow_from_directory(
    train_data_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='sparse',  
    shuffle=False
)

Found 10175 images belonging to 4 classes.


In [7]:
#running the trianing models 
predictions_model1_train = model1.predict(train_generator)
predictions_model2_train = model2.predict(train_generator)
predictions_model3_train = model3.predict(train_generator)

318/318 [==============================] - 872s 3s/step


In [8]:
# Making the labels for the train set
train_labels = train_generator.labels

# Ensembling the predictions on the train set
ensemble_predictions_train = np.argmax((predictions_model1_train + predictions_model2_train + predictions_model3_train), axis=1)

ensemble_accuracy_train = accuracy_score(train_labels, ensemble_predictions_train)
print("Ensemble Accuracy on Train Set:", ensemble_accuracy_train)

Ensemble Accuracy on Train Set: 0.3307125307125307


In [9]:
#test preprocess 
test_data_gen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_data_gen.flow_from_directory(
    test_data_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='sparse',  
    shuffle=False
)

Found 2177 images belonging to 4 classes.


In [10]:
#running the test data 
predictions_model1_test = model1.predict(test_generator)
predictions_model2_test = model2.predict(test_generator)
predictions_model3_test = model3.predict(test_generator)

69/69 [==============================] - 162s 2s/step


In [11]:
#making the labels 
test_labels = test_generator.labels

# Ensembling the predictions
ensemble_predictions_test = np.argmax((predictions_model1_test + predictions_model2_test + predictions_model3_test), axis=1)

ensemble_accuracy_test = accuracy_score(test_labels, ensemble_predictions_test)
print("Ensemble Accuracy on Test Set:", ensemble_accuracy_test)

Ensemble Accuracy on Test Set: 0.3330271015158475


In [12]:
#setting x and y 
X_train_gb = ensemble_predictions_train
y_train_gb = train_labels
X_test_gb = ensemble_predictions_test
y_test_gb = test_labels

In [13]:
# reshaping into a matrix 
X_train_gb = X_train_gb.reshape(-1, 1)

#verifying it is an array and doing a grid search 
assert y_train_gb.ndim == 1
gb_classifier = GradientBoostingClassifier(random_state=42)
param_grid = {
    'n_estimators': [2, 3, 5],
    'learning_rate': [5, 3, 1],
    'max_depth': [2, 3]
}
grid_search = GridSearchCV(gb_classifier, param_grid, cv=5)
grid_search.fit(X_train_gb, y_train_gb)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print("Best Hyperparameters:", best_params)

#take the best model and train it 
best_model.fit(X_train_gb, y_train_gb)

Best Hyperparameters: {'learning_rate': 1, 'max_depth': 2, 'n_estimators': 2}


GradientBoostingClassifier(learning_rate=1, max_depth=2, n_estimators=2,
                           random_state=42)

In [14]:
# reshape X_test_gb to an array
X_test_gb = X_test_gb.reshape(-1, 1)

# predictions
X_test_gb = X_test_gb.reshape(-1, 1)
y_val_pred = best_model.predict(X_test_gb)
val_accuracy = accuracy_score(y_test_gb, y_val_pred)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.34726688102893893


In [15]:
#saving the model 
model_filename = '../models/gb.pkl'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

Model saved to ../models/gb.pkl
